In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Define path to dataset
data_dir = "/kaggle/input/indian-food-classification/Food Classification"

In [ ]:
# Define image size and batch size
image_size = (160, 160)  # Increase to 160x160 for better feature learning
batch_size = 32

In [ ]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,  # 80-20 split for training and validation
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
# Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Set as training data
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)


In [ ]:
from tensorflow.keras.applications import MobileNetV2

# Specify the path to the weights file
weights_path = '/kaggle/input/h5/pytorch/default/1/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_160_no_top.h5'

# Load MobileNetV2 with the local weights, excluding the top layers
base_model = MobileNetV2(weights=weights_path, include_top=False, input_shape=(160, 160, 3))


In [ ]:
# Freeze the base model to retain pre-trained weights during initial training
base_model.trainable = False

In [ ]:
# Add custom layers on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
# Initial training with frozen base model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)


In [ ]:
# Evaluate the model on validation data
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")


In [ ]:
# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[-30:]:  # Unfreeze last 30 layers
    layer.trainable = True

In [ ]:
# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)


In [ ]:
# Evaluate the model on validation data
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")


In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(fine_tune_history.history['accuracy'], label='Fine-tune Train Accuracy')
plt.plot(fine_tune_history.history['val_accuracy'], label='Fine-tune Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(fine_tune_history.history['loss'], label='Fine-tune Train Loss')
plt.plot(fine_tune_history.history['val_loss'], label='Fine-tune Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.show()

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to load and preprocess a single image
def prepare_image(img_path, target_size=(160, 160)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize
    return img_array

# Path to a new test image
img_path = '/kaggle/input/kqaaaaat/kat.jpg'

# Prepare the image
img = prepare_image(img_path)

# Make prediction
predictions = model.predict(img)
predicted_class = np.argmax(predictions)  # Index of the highest probability class
print(f"Predicted Class Index: {predicted_class}")


In [ ]:
# Assuming the prediction code is similar to this:
predictions = model.predict(img)
predicted_class_index = np.argmax(predictions)  # Index of the highest probability class

# Map the predicted index to the class name using the class_indices
class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}  # Reverse the dictionary to map index to label

# Get the predicted class name
predicted_class_name = class_labels[predicted_class_index]
print(f"Predicted Food Class: {predicted_class_name}")


In [ ]:
# Save the model
model_save_path = '/kaggle/working/food_classification_model.h5'
model.save(model_save_path)
print(f"Model saved to {model_save_path}")


In [5]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('/content/food_classification_model.h5')

In [7]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to load and preprocess a single image
def prepare_image(img_path, target_size=(160, 160)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize
    return img_array

# Path to a new test image
img_path = '/content/7f50fc7a906cade219ee211200f3fc48.jpg'

# Prepare the image
img = prepare_image(img_path)

# Make prediction
predictions = loaded_model.predict(img)
predicted_class = np.argmax(predictions)  # Index of the highest probability class
print(f"Predicted Class Index: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicted Class Index: 4


In [9]:
pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 14.9 MB/s eta 0:00:00


In [11]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Define fuzzy variables
prob1 = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'prob1')
prob2 = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'prob2')
confidence = ctrl.Consequent(np.arange(0, 1.1, 0.1), 'confidence')

# Membership functions
prob1['low'] = fuzz.trimf(prob1.universe, [0, 0, 0.5])
prob1['medium'] = fuzz.trimf(prob1.universe, [0.2, 0.5, 0.8])
prob1['high'] = fuzz.trimf(prob1.universe, [0.5, 1, 1])

prob2['low'] = fuzz.trimf(prob2.universe, [0, 0, 0.5])
prob2['medium'] = fuzz.trimf(prob2.universe, [0.2, 0.5, 0.8])
prob2['high'] = fuzz.trimf(prob2.universe, [0.5, 1, 1])

confidence['low'] = fuzz.trimf(confidence.universe, [0, 0, 0.5])
confidence['medium'] = fuzz.trimf(confidence.universe, [0.2, 0.5, 0.8])
confidence['high'] = fuzz.trimf(confidence.universe, [0.5, 1, 1])

# Define fuzzy rules
rule1 = ctrl.Rule(prob1['high'] & prob2['low'], confidence['high'])
rule2 = ctrl.Rule(prob1['medium'] & prob2['medium'], confidence['medium'])
rule3 = ctrl.Rule(prob1['low'] & prob2['high'], confidence['low'])

# Create and simulate fuzzy control system
confidence_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
confidence_sim = ctrl.ControlSystemSimulation(confidence_ctrl)

# Modify prediction function
def fuzzy_predict(predictions):
    top_2_indices = np.argsort(predictions[0])[-2:]  # Get indices of top 2 predictions
    top_2_probs = predictions[0][top_2_indices]

    # Pass probabilities to fuzzy system
    confidence_sim.input['prob1'] = top_2_probs[0]
    confidence_sim.input['prob2'] = top_2_probs[1]
    confidence_sim.compute()

    # Get fuzzy confidence score
    fuzzy_confidence = confidence_sim.output['confidence']

    # Decide final prediction
    if fuzzy_confidence > 0.7:  # High confidence threshold
        return top_2_indices[0]  # Take highest probability class
    elif fuzzy_confidence > 0.4:  # Medium confidence threshold
        return f"Ambiguous: {top_2_indices}"  # Indicate ambiguity
    else:
        return top_2_indices[1]  # Take second-highest probability class

# Example usage
predictions = loaded_model.predict(img)
result = fuzzy_predict(predictions)
print(f"Fuzzy Prediction: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Fuzzy Prediction: 4
